# GP Cartpole Service

This notebook uses the functions from the `gprest` module to enable genetic programming for REST service behaviors. It interacts with REST server stubs created by the OpenAPI generator.

The previous notebook [`gp_cartpole_server.ipynb`](https://github.com/cdeck3r/DRL4REST/notebooks/gp_cartpole_server.ipynb) shows the GP approach applied to the [cartpole](https://en.wikipedia.org/wiki/Inverted_pendulum) example. The functions from this notebook are now a part of the `gprest` module.

This notebook reproduces the results from `gp_cartpole_server.ipynb`.

## Install required Modules

In [1]:
%%bash
# define project environment
PROJECT_DIR="/DRL4REST"
OPENAPI_SERVER_DIR="$PROJECT_DIR/openapi/cartpole/python-flask"

# install requirements
cd "$OPENAPI_SERVER_DIR" || exit
pip install -r requirements.txt || exit
pip install -r test-requirements.txt || exit

# install other requirements
pip install Werkzeug==0.16.1 || exit

Ignoring connexion: markers 'python_version == "3.5"' don't match your environment
Ignoring connexion: markers 'python_version == "3.4"' don't match your environment
Ignoring connexion: markers 'python_version <= "2.7"' don't match your environment
  Created wheel for flask-testing: filename=Flask_Testing-0.6.1-py3-none-any.whl size=7639 sha256=17361730657d8a29a2e65894451cfdb2ced7ea845ee05254e90c4e62aeecfa74
  Stored in directory: /home/jovyan/.cache/pip/wheels/d3/a7/7f/effdb220813fd331ca8d1961b4fd629fa6dcd54c9cdc83eaeb
Successfully built flask-testing
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1


In [2]:
import random
import numpy
from functools import partial

In [3]:
# DEAP: python genetic algorithm library
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [4]:
# set path to import code generated by OpenAPI
import sys
sys.path.append("/DRL4REST/openapi/cartpole/python-flask")
sys.path.append("/DRL4REST/src")

In [5]:
from cartpole.gprest.cartpole_server import CartpoleServer
from cartpole.gprest.monkey_patching import MonkeyPatching
from cartpole.gprest.gp_test_default_controller import GP_TestDefaultController

## Genetic Program Implementation

The GP program bases upon the Artificial Ant Problem from the [DEAP example site](https://deap.readthedocs.io/en/master/examples/gp_ant.html). The implementation consists of

* generic functions for the control flow
* controller functions interacting with the ServerModel
* `evaluate()` function to report the controller's fitness

### Generic Functions

These are generic functions, e.g. for control flow and concatenating functions with each other.

In [6]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def _if_then_else(condition, out1, out2):
    if condition() is not None:
        out1()
    else:
        out2()

def if_then_else(condition, out1, out2):
    return partial(_if_then_else, condition, out1, out2)

def _if_then(condition, out1):
    if condition() is not None:
        out1()
        
def if_then(condition, out1):
    return partial(_if_then, condition, out1)

### Controller Functions

These are the functions the GP algorithm strings together utilizing the generic functions from above. The created program forms an individual and is tested by the `evaluate()` function. The controller functions utilize the CartpoleServer's CRUD functions.

In [7]:
cps = CartpoleServer
cps.reset()

In [8]:
# let's create the set of all functions DEAP shall work with
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(if_then_else, 3)
pset.addPrimitive(if_then, 2)

pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)

pset.addTerminal(cps.create_cart)
pset.addTerminal(cps.read_cart)
pset.addTerminal(cps.update_cart)
pset.addTerminal(cps.delete_cart)

In [9]:
# cart_get() controller must return something when done
# this function becomes the new root for the GP tree created from the pset above
# it runs the GP tree first and returns the Cart afterwards
def return_cart_get(left):
    def run_first_return_second(f1,f2):
        f1()
        return f2()
    
    return partial(run_first_return_second, left, cps.read_cart)

### `evaluate()` Function

It patches the endpoint with the new controller function and runs the controller unittest. Evaluation reports the controller's fitness to guide the next program evolution.

In [10]:
class GP_Controller:
    """Static class which encapsulates the gp_controller for execution
    
    """
    
    _gp_controller = None
    
    @classmethod
    def set_controller_func(cls, func):
        cls._gp_controller = func

    @staticmethod
    def gp_controller_func():
        """Runs the gp_controller.

        gp_controller_func is called from a different scope, so it needs to 
        search through all imported modules for the GP_Controller class in
        order to access the class variable _gp_controller.

        CAUTION: 
        This is fragile. It finds the first occurance of the GP_Controller
        within all imported modules.

        DEFAULT:
        It searches the __main__ module only for the GP_Controller class.
        """

        # This is the generic search routine
        """
        gpc_pointer = None
        for m_name in sys.modules:
            try:
                gpc_pointer = getattr(sys.modules[m_name], 'GP_Controller')
                break
            except AttributeError:
                continue
        return gpc_pointer._gp_controller()
        """
        # finds the GP_Controller class in the __main__ module
        return getattr(sys.modules['__main__'], 'GP_Controller')._gp_controller()
    
gpc = GP_Controller

In [11]:
import types
import json

# evaluate()
def evaluate_cart_get(individual):
   
    def add_func2pset(expr, pset, func, num_args):
        pset_cloned = toolbox.clone(pset)
        pset_cloned.addPrimitive(func, num_args)
        prim_func = gp.Primitive(func.__name__, [object], object)
        expr_func = [prim_func] + expr
        return expr_func, pset_cloned

    # we add the return_cart_get() als new root of the individual
    # togehter they form the gp_controller

    assert isinstance(return_cart_get, types.FunctionType), 'return_cart_get() not defined'
    
    # 1. dismantle the individual to get the expressions
    expr = list(individual)

    # 2. add return_cart_get als new root
    expr_return, pset_return = add_func2pset(expr=expr,
                                             pset=pset,
                                             func=return_cart_get, 
                                             num_args=1)
    # 3. rebuild individual (type: PrimitiveTree) from expressions 
    individual_return = gp.PrimitiveTree(expr_return)
    
    # 4. .. and compile tree to functional Python code 
    gp_controller = gp.compile(individual_return, pset_return)

    # store the controller
    gpc.set_controller_func(gp_controller)
    #gpc.set_controller_func(testcase_all_correct)

    # Replace the default controller with the gp_controller 
    url_path = '/api/v1/cart'
    gp_test = GP_TestDefaultController()
    ret = gp_test.endpoint_config(url_path, 'get', gpc.gp_controller_func)
    # ... and test
    cps.reset()
    gp_test.reset_score()
    
    #gp_test.test_cart_get()
    # run the test several times and expect the same as the first one
    for i in range(5):
        gp_test.safe_test_cart_get()
    
    # retrieve score 
    fitness = gp_test.score
    
    # for debugging
    if fitness == -1: 
        # store individual
        gp_program = str(individual)
        # store _prev_responses
        all_responses = gp_test._prev_responses
        with open("/home/jovyan/work/gp_program.json", "a") as f:
            json_gp_program = json.dumps(gp_program)
            json_all_responses = json.dumps(all_responses)
            f.write(str(fitness))
            f.write(json_gp_program)
            f.write(json_all_responses)
            f.write('\n\n')
    
    return fitness,

## Assemble and Configure the GP

We configure the GP to solve a minimization problem, `FitnessMin`. The `evaluate()` function returns the cumulative sum of the responses' status codes when testing the controller as fitness. 

In [12]:
# configure the parameters
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

# Attribute generator
toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

Register the GP operators. They mate, mutate and change in various ways the individuals made of programs. Further individuals are the result from the GP algorithm using these GP operators.

In [13]:
toolbox.register("evaluate", evaluate_cart_get)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

## Run the GP Algorithm

In [14]:
random.seed(123)

pop = toolbox.population(n=300)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 2, stats, halloffame=hof)

gen	nevals	avg    	std    	min	max 
0  	300   	3329.67	1991.85	199	4999
1  	177   	1601.67	1517.72	199	4999
2  	180   	1391   	1532.56	199	4999


## Display and Plot

Plot the individual. It is a tree structure contained in the halloffame `hof` object.

`hof[0]` indicates the individual tree structure.

In [15]:
str(hof[0])

'if_then_else(read_cart, read_cart, create_cart)'